In [ ]:
import pandas as pd
import numpy as np
import os

import pandas as pd
import numpy as np
from collections import Counter
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import math
from sklearn.preprocessing import MinMaxScaler
import scipy
from sklearn import metrics

import numpy as np
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr

import seaborn as sns
import matplotlib.pyplot as plt
import shap

from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from scipy.stats import linregress
from scipy import stats
from sklearn.utils import resample

In [ ]:
import numpy as np
from scipy.stats import percentileofscore
from sklearn.utils import check_random_state
from itertools import islice

def permtest(x, y, statistic="mean", max_samples=5000, random_state=42):
    """
    Conducts a permutation test between two independent samples.
    Parameters
    ----------
    x : ndarray
        First set of datapoints.
    y : ndarray
        Second set of datapoints.
    statistic : str or callable, optional
        Function that takes in samples x and y and reports
        statistic of interest. By default, "mean" reports
        the difference of sample means. List of default
        options are ("mean", "median").
    max_samples : int, optional.
        Maximum number of label permutations to try.
    random_state : np.random.RandomState, int, or None.
        If specified, used to seed the random number generator to shuffle the ordering of the datapoints.
    """

    # Initialize random state and function of interest
    rs = check_random_state(random_state)
    stat_func = _get_stat_func(statistic)

    # Concatenate samples
    xy = np.concatenate((x, y))
    n_x = x.size

    # Observed statistic
    observed_stat = stat_func(x, y)

    # Set the number of permutations to compute
    n_perms = max(max_samples, 100000)  # Limit the number of permutations to a manageable number (e.g., 10000)

    # Allocate space for computed statistics
    shuffled_stats = np.empty(n_perms)

    # Print coverage
    print(f"Computing permutations...")

    # Sampling random permutations
    for i in range(n_perms):
        rs.shuffle(xy)
        x_ = xy[:n_x]
        y_ = xy[n_x:]
        shuffled_stats[i] = stat_func(x_, y_)

    # Compute a two-sided p-value. We take the smallest
    # percentile and then multiply by two.
    pval = 2 * min(
        percentileofscore(shuffled_stats, observed_stat, kind="rank") / 100,
        1 - percentileofscore(shuffled_stats, observed_stat, kind="rank") / 100
    )

    return pval

def _get_stat_func(name_or_func):
    """
    Instantiates functions that compute default statistics of interest.
    """
    # If specified function is callable
    if not isinstance(name_or_func, str):
        if callable(name_or_func):
            return name_or_func
        else:
            raise ValueError(
                "`statistic` should be a string like ('mean', 'median')"
                " or a function that takes in samples x, y and returns"
                " the statistic of interest."
            )

    # Default functions
    if name_or_func == "mean":
        return lambda x, y: np.mean(x) - np.mean(y)
    elif name_or_func == "median":
        return lambda x, y: np.median(x) - np.median(y)
    else:
        raise ValueError("Did not recognize statistic.")



In [ ]:
import numpy as np

def pooled(x, y, func):
    nx = len(x)
    ny = len(y)
    return np.sqrt(((nx - 1) * func(x) ** 2 + (ny - 1) * func(y) ** 2) / (nx + ny - 2))

def hdmedian(x):
    return np.median(x)

def hdmad(x):
    return 1.4826 * hdmedian(np.abs(x - hdmedian(x)))

def phdmad(x, y):
    return pooled(x, y, hdmad)

def gamma_effect_size(x, y, prob):
    hdquantile_x = np.percentile(x, prob * 100)
    hdquantile_y = np.percentile(y, prob * 100)
    return (hdquantile_y - hdquantile_x) / phdmad(x, y)



In [ ]:
def remove_outliers_iqr_01(data):

    data = np.array(data)

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1


    ir_mult = 0.1
    lower_bound = Q1 - ir_mult * IQR
    upper_bound = Q3 + ir_mult * IQR

    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    
    return filtered_data

def remove_outliers_iqr_05(data):

    data = np.array(data)

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1


    ir_mult = 0.5
    lower_bound = Q1 - ir_mult * IQR
    upper_bound = Q3 + ir_mult * IQR

    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    
    return filtered_data

def remove_outliers_iqr_10(data):

    data = np.array(data)

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1

    ir_mult = 1.0
    lower_bound = Q1 - ir_mult * IQR
    upper_bound = Q3 + ir_mult * IQR

    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    
    return filtered_data

def cliff_delta(group1, group2):

    group1_ = remove_outliers_iqr_01(group1)
    group2_ = remove_outliers_iqr_01(group2)
    
    sub_sample_size = np.min([int(group1_.shape[0]*0.7), int(group2_.shape[0]*0.7)])


    delta_l = []
    for i in range(10):
       
        group1 = resample(group1_, n_samples=sub_sample_size, replace=False, random_state=i)
        group2 = resample(group2_, n_samples=sub_sample_size, replace=False, random_state=i)
    
        group1_exp = group1[:, np.newaxis]  
        group2_exp = group2[np.newaxis, :]  
    
        greater = np.sum(group1_exp > group2_exp)
        less = np.sum(group1_exp < group2_exp)
    
        n1, n2 = len(group1), len(group2)
        delta = (greater - less) / (n1 * n2)

    delta_l.append(delta)

    print('N', n1+n2, n1, n2)

    return (np.round(np.mean(delta_l), 4), np.round(np.std(delta_l)*2.58,100))

def cliff_delta05(group1, group2):

    group1_ = remove_outliers_iqr_05(group1)
    group2_ = remove_outliers_iqr_05(group2)
    
    sub_sample_size = np.min([int(group1_.shape[0]*0.7), int(group2_.shape[0]*0.7)])


    delta_l = []
    for i in range(10):
       
        group1 = resample(group1_, n_samples=sub_sample_size, replace=False, random_state=i)
        group2 = resample(group2_, n_samples=sub_sample_size, replace=False, random_state=i)
    
        group1_exp = group1[:, np.newaxis] 
        group2_exp = group2[np.newaxis, :]  
    
        greater = np.sum(group1_exp > group2_exp)
        less = np.sum(group1_exp < group2_exp)
    
        n1, n2 = len(group1), len(group2)
        delta = (greater - less) / (n1 * n2)

    delta_l.append(delta)

    print('N', n1+n2, n1, n2)

    return (np.round(np.mean(delta_l), 4), np.round(np.std(delta_l)*2.58,100))

def cliff_delta10(group1, group2):

    group1_ = remove_outliers_iqr_10(group1)
    group2_ = remove_outliers_iqr_10(group2)
    
    sub_sample_size = np.min([int(group1_.shape[0]*0.7), int(group2_.shape[0]*0.7)])


    delta_l = []
    for i in range(10):
       
        group1 = resample(group1_, n_samples=sub_sample_size, replace=False, random_state=i)
        group2 = resample(group2_, n_samples=sub_sample_size, replace=False, random_state=i)
    
        group1_exp = group1[:, np.newaxis]  
        group2_exp = group2[np.newaxis, :]   
    
        greater = np.sum(group1_exp > group2_exp)
        less = np.sum(group1_exp < group2_exp)
    
        n1, n2 = len(group1), len(group2)
        delta = (greater - less) / (n1 * n2)

    delta_l.append(delta)

    print('N', n1+n2, n1, n2)

    return (np.round(np.mean(delta_l), 4), np.round(np.std(delta_l)*2.58,100))


def calculate_z_from_u(group1, group2, u_statistic):
    group1 = remove_outliers_iqr(group1)
    group2 = remove_outliers_iqr(group2)


    n1, n2 = len(group1), len(group2)

    mu_u = (n1 * n2) / 2
    sigma_u = np.sqrt((n1 * n2 * (n1 + n2 + 1)) / 12)

    z_value = (u_statistic - mu_u) / sigma_u

    r_value = z_value / np.sqrt(n1 + n2)

    return r_value


def rank_biserial_from_u(group1, group2):
    group1 = remove_outliers_iqr_01(group1)
    group2 = remove_outliers_iqr_01(group2)

    u_statistic, _ = stats.mannwhitneyu(group1, group2, alternative='two-sided')

    n1, n2 = len(group1), len(group2)

    r_biserial = 1 - (2 * u_statistic) / (n1 * n2)

    return r_biserial

def rank_biserial_from_u10(group1, group2):
    group1 = remove_outliers_iqr_10(group1)
    group2 = remove_outliers_iqr_10(group2)

    u_statistic, _ = stats.mannwhitneyu(group1, group2, alternative='two-sided')

    n1, n2 = len(group1), len(group2)

    r_biserial = 1 - (2 * u_statistic) / (n1 * n2)

    return r_biserial

## SES with education

In [ ]:
pd_merge_wo_nan =  pd.read_excel('data/SES_data.xlsx')
pd_merge_wo_nan = pd_merge_wo_nan.dropna(subset=['SES_wo_w'])
pd_merge_wo_nan.shape

In [ ]:
results_merge_df_all = pd_merge_wo_nan.copy()



display(results_merge_df_all.shape)

bef = results_merge_df_all.shape[0]

Q1 = results_merge_df_all['GAP_corrected'].quantile(0.25)
Q3 = results_merge_df_all['GAP_corrected'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

from scipy import stats

results_merge_df_all['z_score'] = stats.zscore(results_merge_df_all['GAP_corrected'])

outliers = results_merge_df_all[(results_merge_df_all['GAP_corrected'] < lower_bound) | (results_merge_df_all['GAP_corrected'] > upper_bound)]
outliers2 = results_merge_df_all[(results_merge_df_all['z_score'] > 3) | (results_merge_df_all['z_score'] < -3)]


results_merge_df_all = results_merge_df_all[(results_merge_df_all['GAP_corrected'] >= lower_bound) & (results_merge_df_all['GAP_corrected'] <= upper_bound)]

results_merge_df_all = results_merge_df_all.dropna(subset=['SES_wo_w', 'GAP_corrected'])


gb_region = results_merge_df_all.groupby(['Region_x'])['GAP_corrected'].agg(['mean', 'std', 'median']).reset_index()



aft = results_merge_df_all.shape[0]

display(results_merge_df_all.shape)
display(100*aft/bef)


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()



from scipy.stats import linregress

plt.figure(figsize=(6.7,6.7))

plt.subplot(3,3,1)
sns.scatterplot(x='SES_wo_w', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['SES_wo_w'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['SES_wo_w'], intercept + slope * results_merge_df_all['SES_wo_w'], color='blue')
plt.ylim([-12, 12])
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')

In [ ]:
results_merge_df_all = pd_merge_wo_nan.copy()

results_merge_df_all = results_merge_df_all.dropna(subset=['gender_equal_l',
                                                            'Migration',
                                                            'Polution_conc_inv',
                                                            'Eq',
                                                            'free_parties_l',
                                                            'inclu_suff_est',
                                                            'cred_elect_est',
                                                            'local_dem_est'])



display(results_merge_df_all.shape)

bef = results_merge_df_all.shape[0]

Q1 = results_merge_df_all['GAP_corrected'].quantile(0.25)
Q3 = results_merge_df_all['GAP_corrected'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

from scipy import stats

results_merge_df_all['z_score'] = stats.zscore(results_merge_df_all['GAP_corrected'])

outliers = results_merge_df_all[(results_merge_df_all['GAP_corrected'] < lower_bound) | (results_merge_df_all['GAP_corrected'] > upper_bound)]
outliers2 = results_merge_df_all[(results_merge_df_all['z_score'] > 3) | (results_merge_df_all['z_score'] < -3)]


results_merge_df_all = results_merge_df_all[(results_merge_df_all['GAP_corrected'] >= lower_bound) & (results_merge_df_all['GAP_corrected'] <= upper_bound)]

results_merge_df_all.reset_index(inplace = True, drop = True)

aft = results_merge_df_all.shape[0]

display(results_merge_df_all.shape)
display(100*aft/bef)


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

from sklearn.preprocessing import MinMaxScaler

vars_physical = ['gender_equal_l', 'Migration', 'Polution_conc_inv', 'Eq']

vars_extended = ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est']

scaler = MinMaxScaler()
results_merge_df_all[vars_physical] = scaler.fit_transform(results_merge_df_all[vars_physical])
results_merge_df_all['physical'] = results_merge_df_all[vars_physical].mean(axis=1)

results_merge_df_all[vars_extended] = scaler.fit_transform(results_merge_df_all[vars_extended])
results_merge_df_all['extended'] = results_merge_df_all[vars_extended].mean(axis=1)

total_exposomes = vars_physical + vars_extended
results_merge_df_all[total_exposomes] = scaler.fit_transform(results_merge_df_all[total_exposomes])
results_merge_df_all['total_exposomes'] = results_merge_df_all[total_exposomes].mean(axis=1)


from scipy.stats import linregress

plt.figure(figsize=(6.7,2.7))

plt.subplot(1,3,1)
sns.scatterplot(x='physical', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['physical'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['physical'], intercept + slope * results_merge_df_all['physical'], color='blue')
#plt.title(f'r = {r_value:.2f} p = {p_value:.2e}')

r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')

plt.subplot(1,3,2)
sns.scatterplot(x='extended', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['extended'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['extended'], intercept + slope * results_merge_df_all['extended'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')

plt.subplot(1,3,3)
sns.scatterplot(x='total_exposomes', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['total_exposomes'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['total_exposomes'], intercept + slope * results_merge_df_all['total_exposomes'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')

plt.tight_layout()

In [ ]:
results_merge_df_all = pd_merge_wo_nan.copy()

results_merge_df_all = results_merge_df_all.dropna(subset=['gender_equal_l',
                                                            'Migration',
                                                            'Polution_conc_inv',
                                                            'Eq',
                                                            'free_parties_l',
                                                            'inclu_suff_est',
                                                            'cred_elect_est',
                                                            'local_dem_est'])

display(results_merge_df_all.shape)

bef = results_merge_df_all.shape[0]

Q1 = results_merge_df_all['GAP_corrected'].quantile(0.25)
Q3 = results_merge_df_all['GAP_corrected'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

from scipy import stats

results_merge_df_all['z_score'] = stats.zscore(results_merge_df_all['GAP_corrected'])

outliers = results_merge_df_all[(results_merge_df_all['GAP_corrected'] < lower_bound) | (results_merge_df_all['GAP_corrected'] > upper_bound)]
outliers2 = results_merge_df_all[(results_merge_df_all['z_score'] > 3) | (results_merge_df_all['z_score'] < -3)]


results_merge_df_all = results_merge_df_all[(results_merge_df_all['GAP_corrected'] >= lower_bound) & (results_merge_df_all['GAP_corrected'] <= upper_bound)]


results_merge_df_all.reset_index(inplace = True, drop = True)

aft = results_merge_df_all.shape[0]

display(results_merge_df_all.shape)
display(100*aft/bef)


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


from scipy.stats import linregress

plt.figure(figsize=(6.7,6.7))

plt.subplot(3,3,1)
sns.scatterplot(x='gender_equal_l', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['gender_equal_l'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['gender_equal_l'], intercept + slope * results_merge_df_all['gender_equal_l'], color='blue')
#plt.title(f'r = {r_value:.2f} p = {p_value:.2e}')

r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')

plt.subplot(3,3,2)
sns.scatterplot(x='Migration', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['Migration'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['Migration'], intercept + slope * results_merge_df_all['Migration'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')



plt.subplot(3,3,3)
sns.scatterplot(x='Polution_conc_inv', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['Polution_conc_inv'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['Polution_conc_inv'], intercept + slope * results_merge_df_all['Polution_conc_inv'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')


plt.subplot(3,3,4)
sns.scatterplot(x='Eq', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['Eq'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['Eq'], intercept + slope * results_merge_df_all['Eq'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')


plt.subplot(3,3,5)
sns.scatterplot(x='representation_l', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['representation_l'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['representation_l'], intercept + slope * results_merge_df_all['representation_l'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')


plt.subplot(3,3,6)
sns.scatterplot(x='free_parties_l', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['free_parties_l'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['free_parties_l'], intercept + slope * results_merge_df_all['free_parties_l'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')



plt.subplot(3,3,7)
sns.scatterplot(x='inclu_suff_est', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['inclu_suff_est'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['inclu_suff_est'], intercept + slope * results_merge_df_all['inclu_suff_est'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')


plt.subplot(3,3,8)
sns.scatterplot(x='cred_elect_est', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['cred_elect_est'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['cred_elect_est'], intercept + slope * results_merge_df_all['cred_elect_est'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')


plt.subplot(3,3,9)
sns.scatterplot(x='local_dem_est', y='GAP_corrected', data=results_merge_df_all, alpha=0.7, color='gray')
slope, intercept, r_value, p_value, std_err = linregress(results_merge_df_all['local_dem_est'], results_merge_df_all['GAP_corrected'])
plt.plot(results_merge_df_all['local_dem_est'], intercept + slope * results_merge_df_all['local_dem_est'], color='blue')
r_value1 = np.abs(r_value)
d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
plt.title( f'd-cohen={d_cohen:.2f} \nr={r_value:.2f} p={p_value:.2e}')

plt.tight_layout()

### Co-variance

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import scipy.stats as stats
import pingouin as pg

subset=['gender_equal_l',
        'Migration',
        'Polution_conc_inv',
        'Eq',
        'free_parties_l',
        'inclu_suff_est',
        'cred_elect_est',
        'local_dem_est', 'total_exposomes', 'extended', 'physical', 'GAP_corrected', 'representation_l', 'SES_wo_w']



combined_exposomes_comb_ = [['total_exposomes', 'SES_wo_w'],
                            ['physical', 'SES_wo_w'],
                            ['extended', 'SES_wo_w'],
                            ['gender_equal_l', 'SES_wo_w'],
                            ['Migration', 'SES_wo_w'],
                            ['Eq', 'SES_wo_w'],
                            ['Polution_conc_inv', 'SES_wo_w'],
                             ['representation_l', 'SES_wo_w'],
                            ['free_parties_l', 'SES_wo_w'],
                            ['inclu_suff_est', 'SES_wo_w'],
                            ['cred_elect_est', 'SES_wo_w'],
                            ['local_dem_est', 'SES_wo_w'],
                          ]




combined_exposomes_comb_results = {}

for i in combined_exposomes_comb_:

    df = pd_merge_wo_nan.copy()


    vars_physical = ['gender_equal_l', 'Migration', 'Polution_conc_inv', 'Eq']
    
    vars_extended = ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est']
    

    scaler = MinMaxScaler()
    df[vars_physical] = scaler.fit_transform(df[vars_physical])
    df['physical'] = df[vars_physical].mean(axis=1)
    
    df[vars_extended] = scaler.fit_transform(df[vars_extended])
    df['extended'] = df[vars_extended].mean(axis=1)
    
    total_exposomes = vars_physical + vars_extended
    df[total_exposomes] = scaler.fit_transform(df[total_exposomes])
    df['total_exposomes'] = df[total_exposomes].mean(axis=1)

    df =  df[subset]

    if( i[0] == 'Polution_conc_inv'):
        Q1 = df[i[0]].quantile(0.25)
        Q3 = df[i[0]].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        df = df[(df[i[0]] >= lower_bound) & (df[i[0]] <= upper_bound)]

    X_train, X_test, y_train, y_test = train_test_split(df[subset], df['GAP_corrected'], test_size=0.1, random_state=42)

    X_train.reset_index(inplace= True, drop = True)
    X = X_train[i]
    y = X_train['GAP_corrected']
    
    X = sm.add_constant(X)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=40)
    
    y_labels = []
    y_predicts = []
    
    
    iter_ = 0
    for train_index, test_index in kf.split(X):
    
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
        y_train, y_test = y[train_index], y[test_index]
    
        model = sm.OLS(y_train, X_train).fit()
    
        predicted_values = model.predict(X_test)
    
    
        y_labels.extend(list(y_test))
        y_predicts.extend(list(predicted_values))


    model = sm.OLS(y, X).fit()
    

    coefficients = model.params
    t_values = model.tvalues
    p_values = model.pvalues
    
    results_summary = pd.DataFrame({
        'Predictor': coefficients.index,  
        'Coefficient': coefficients.values,  
        't-Value': t_values.values, 
        'p-Value': p_values.values 
    })
     
    
    r_squared = r2_score(y_labels, y_predicts)

    import pandas as pd
    import numpy as np
    

    partial_corr = pg.partial_corr(data=df, x=i[0], y='GAP_corrected', covar=i[1])

    r_value = partial_corr.loc['pearson', 'r']
    p_value = partial_corr.loc['pearson', 'p-val']
    r_value_pred, p_value_pred = stats.pearsonr(y_labels, y_predicts)

    r_value1 = np.abs(r_value)
    d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)

    results_summary = pd.DataFrame({
        'Predictor': coefficients.index,  
        'Coefficient': coefficients.values,  
        't-Value': t_values.values, 
        'p-Value': p_values.values, 
        'd_cohen': d_cohen, 
        'p_value_glob': p_value  
    })

    combined_exposomes_comb_results['_'.join(i)] = [y_labels, y_predicts, r_value, p_value, results_summary, r_value_pred, p_value_pred]
    
    print(f"r en el conjunto de prueba: {'_'.join(i):s} {r_value:.3f}")

### Statistics

#### GDP

In [ ]:
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

df1 = pd_merge_wo_nan.copy()



Q1 = df1['GAP_corrected'].quantile(0.25)
Q3 = df1['GAP_corrected'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.0 * IQR
upper_bound = Q3 + 1.0 * IQR

from scipy import stats


df1 = df1[(df1['GAP_corrected'] >= lower_bound) & (df1['GAP_corrected'] <= upper_bound)]


bins = [0, df1['GDP_percapita'].median(), float('inf')]  
labels = ['L_GDP', 'H_GDP']
df1['Group_GDP'] = pd.cut(df1['GDP_percapita'], bins=bins, labels=labels)

df1.dropna(subset = ['Group_GDP'], inplace = True)

df1['SES_b'] = np.where(
    df1['SES'] > df1['SES_wo_w'].median(), 'Hses',
    np.where(
        df1['SES'] <= df1['SES_wo_w'].median(), 'Lses',
        np.nan
    )
)


df1["Group_Combination"] = df1["Group_GDP"].astype(str) + "_" + df1["SES_b"].astype(str)

order = ['H_GDP_Lses', 'L_GDP_Lses', 'L_GDP_Hses',  'H_GDP_Hses']

regions = ['H_GDP_Lses', 'L_GDP_Lses', 'L_GDP_Hses',  'H_GDP_Hses']
mann_whitney_results = []

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = df1[df1['Group_Combination'] == regions[i]]['GAP_corrected']
        region2_data = df1[df1['Group_Combination'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)

        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        #print(f"\nTamaño del efecto de Cohen's d: {cohen_d:.3f}")
        print(f"No parametric cliff_delta: {cliff_d:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)

plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(5, 4))

sns.violinplot(x='Group_Combination', y='GAP_corrected', data=df1, order=order, palette="gray", inner="box")

plt.title('', fontsize=14)
plt.xlabel('Región', fontsize=12)
plt.ylabel('GAP Corrected', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.ylim([-20, 20])
plt.grid(axis='y', linestyle='--', alpha=0.4)

plt.tight_layout()

plt.show()



#### GNI

In [ ]:
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

df1 = pd_merge_wo_nan.copy()



Q1 = df1['GAP_corrected'].quantile(0.25)
Q3 = df1['GAP_corrected'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.0 * IQR
upper_bound = Q3 + 1.0 * IQR

from scipy import stats


df1 = df1[(df1['GAP_corrected'] >= lower_bound) & (df1['GAP_corrected'] <= upper_bound)]


df1['Income_v1'] = df1['Income'] 

df1['SES_b'] = np.where(
    df1['SES'] > df1['SES_wo_w'].median(), 'Hses',
    np.where(
        df1['SES'] <= df1['SES_wo_w'].median(), 'Lses',
        np.nan
    )
)


df1["Group_Combination"] = df1["Income_v1"].astype(str) + "_" + df1["SES_b"].astype(str)

order = ['H_Lses', 'H_Hses', 'L_Lses', 'L_Hses']

regions = ['H_Lses', 'H_Hses', 'L_Lses', 'L_Hses']
mann_whitney_results = []

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = df1[df1['Group_Combination'] == regions[i]]['GAP_corrected']
        region2_data = df1[df1['Group_Combination'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)

        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)

plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(5, 4))

sns.violinplot(x='Group_Combination', y='GAP_corrected', data=df1, order=order, palette="gray", inner="box")

plt.title('', fontsize=14)
plt.xlabel('Región', fontsize=12)
plt.ylabel('GAP Corrected', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.ylim([-20, 20])
plt.grid(axis='y', linestyle='--', alpha=0.4)

plt.tight_layout()

plt.show()

